In [17]:
import random
import numpy as np
from torchvision import datasets, transforms

In [20]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()
    
    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()
    
    x_train = x_train / 255.
    x_test = x_test / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

In this exercise your task is to fill in the gaps in this code by implementing the backpropagation algorithm
Once this is done, you can run the network on the MNIST example and see how it performs. Feel free to play with the parameters.

If you found this task too easy, try to implement a "fully vectorized" version, i.e. one using matrix operations instead of going over examples one by one.

In [21]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def update_mini_batch(self, x_mini_batch, y_mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in zip(x_mini_batch, y_mini_batch):
            delta_nabla_b, delta_nabla_w = self.backprop(x.reshape(784,1), y.reshape(10,1))
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(x_mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]
        
        # Then go forward remembering all values before and after activations
        # in two other array lists
        f = [] #list of values before activation
        g = [x] #list of values after activation function
        for b, w in zip(self.biases, self.weights):
            f.append(np.dot(w, g[-1]) +b)
            g.append(sigmoid(f[-1]))  
        
        # Now go backward from the final cost applying backpropagation
        dL_dg = self.cost_derivative(g[-1], y)
        for l in range(self.num_layers-2, -1, -1):
            delta_nabla_w[l] = (sigmoid_prime(f[l])*dL_dg) @ g[l].T
            delta_nabla_b[l] = np.expand_dims(np.sum(sigmoid_prime(f[l])*dL_dg,  axis = 1), axis = 1)
            dL_dg = np.dot((self.weights[l]).T, sigmoid_prime(f[l])*dL_dg)

        return delta_nabla_b, delta_nabla_w

    def evaluate(self, x_test_data, y_test_data):
        # Count the number of correct answers for test_data
        test_results = [(np.argmax(self.feedforward(x_test_data[i].reshape(784,1))), np.argmax(y_test_data[i]))
                        for i in range(len(x_test_data))]
        # return accuracy
        return np.mean([int(x == y) for (x, y) in test_results])
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)] 
                y_mini_batch = y_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)] 
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test, y_test)))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=50, mini_batch_size=100, eta=3., test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.705
Epoch: 1, Accuracy: 0.7877
Epoch: 2, Accuracy: 0.8356
Epoch: 3, Accuracy: 0.8925
Epoch: 4, Accuracy: 0.9014
Epoch: 5, Accuracy: 0.9077
Epoch: 6, Accuracy: 0.9114
Epoch: 7, Accuracy: 0.9142
Epoch: 8, Accuracy: 0.9171
Epoch: 9, Accuracy: 0.9193
Epoch: 10, Accuracy: 0.921
Epoch: 11, Accuracy: 0.9231
Epoch: 12, Accuracy: 0.9251
Epoch: 13, Accuracy: 0.927
Epoch: 14, Accuracy: 0.9286
Epoch: 15, Accuracy: 0.9293
Epoch: 16, Accuracy: 0.9296
Epoch: 17, Accuracy: 0.9304
Epoch: 18, Accuracy: 0.9314
Epoch: 19, Accuracy: 0.9322
Epoch: 20, Accuracy: 0.9327
Epoch: 21, Accuracy: 0.9335
Epoch: 22, Accuracy: 0.9342
Epoch: 23, Accuracy: 0.9348
Epoch: 24, Accuracy: 0.9353
Epoch: 25, Accuracy: 0.9355
Epoch: 26, Accuracy: 0.9354
Epoch: 27, Accuracy: 0.9355
Epoch: 28, Accuracy: 0.9357
Epoch: 29, Accuracy: 0.9361
Epoch: 30, Accuracy: 0.9371
Epoch: 31, Accuracy: 0.9374
Epoch: 32, Accuracy: 0.9375
Epoch: 33, Accuracy: 0.9375
Epoch: 34, Accuracy: 0.938
Epoch: 35, Accuracy: 0.938
Epoch: 